In [1]:
import json

In [2]:
TAG = 'Defining problem SAR-INDIVIDUAL-PROBLEM ...'

tag_found = False
remove_effect = False
remove_task = False
truncated_data = []
p_plans = []
state_str = ""
with open('printout.txt') as in_file:
    for line in in_file:
        if not tag_found:
            if line.strip() == TAG:
                tag_found = True
        elif line.strip()[0:7] == "state (":
            if line.strip()[-2:] == "))":
                truncated_data.append(line.strip())
            else:
                state_str = line.strip() + " "
        elif state_str:
            if line.strip()[-2:] == "))":
                state_str = state_str + line.strip()
                truncated_data.append(state_str)
                state_str = ""
            elif line.strip()[-1] == "*":
                close_id = line.strip().index("Storing new plan in *plans-found*")
                state_str = "final_" + state_str + line.strip()[0:close_id-1]
                truncated_data.append(state_str)
                state_str = ""
                p_plans.append(truncated_data)
                truncated_data = []
            else:
                state_str = state_str + line.strip() + " "
        elif line.strip()[0:6] == "task (":
            if line.strip()[-1] != ")":
                remove_task = True
        elif line.strip()[0:8] == "effect (":
            if line.strip()[8] == "(" or line.strip()[8:11] == "AND":
                if line.strip()[-2:] != "))":
                    remove_effect = True
        elif remove_task:
            if line.strip()[-1] == ")":
                remove_task = False
        elif remove_effect:
            if line.strip()[-2:] == "))":
                remove_effect = False
        elif line.strip()[0:5] == "Depth":
            comma_id = line.strip().index(',')
            truncated_data.append(line.strip()[0:comma_id])
        elif line.strip()[0:4] == "PDDL":
            close_id = line.strip().index(")")
            truncated_data.append(line.strip()[5:close_id+1])
        elif line.isspace():
            continue
        elif line.strip()[0] == "*":
            continue
        else:
            truncated_data.append(line.strip())

In [3]:
p_plans[0].insert(0,"initial_"+p_plans[0][1])

In [4]:
p_plans[0]

['initial_state ((BUILDING B) (IN V1 R1) (IN V2 R1) (IN V3 R2) (IN V4 R2) (IN V5 R3) (INJURED V2) (INJURED V3) (INJURED V5) (RESCUER T1) (ROOM R1) (ROOM R2) (ROOM R3) (VICTIM V1) (VICTIM V2) (VICTIM V3) (VICTIM V4) (VICTIM V5))',
 'Depth 1',
 'state ((BUILDING B) (IN V1 R1) (IN V2 R1) (IN V3 R2) (IN V4 R2) (IN V5 R3) (INJURED V2) (INJURED V3) (INJURED V5) (RESCUER T1) (ROOM R1) (ROOM R2) (ROOM R3) (VICTIM V1) (VICTIM V2) (VICTIM V3) (VICTIM V4) (VICTIM V5))',
 'action (!ENTER-BUILDING T1 B R1)',
 'state ((BUILDING B) (IN T1 R1) (IN V1 R1) (IN V2 R1) (IN V3 R2) (IN V4 R2) (IN V5 R3) (INJURED V2) (INJURED V3) (INJURED V5) (INSIDE T1 B) (RESCUER T1) (ROOM R1) (ROOM R2) (ROOM R3) (VICTIM V1) (VICTIM V2) (VICTIM V3) (VICTIM V4) (VICTIM V5))',
 'Depth 3',
 'state ((BUILDING B) (IN T1 R1) (IN V1 R1) (IN V2 R1) (IN V3 R2) (IN V4 R2) (IN V5 R3) (INJURED V2) (INJURED V3) (INJURED V5) (INSIDE T1 B) (RESCUER T1) (ROOM R1) (ROOM R2) (ROOM R3) (VICTIM V1) (VICTIM V2) (VICTIM V3) (VICTIM V4) (VICTIM 